### 微调VGG16网络的最后一个卷积块
步骤：  
1) 在已经训练好的基网络上添加自定义网络  
2) 冻结基网络  
3) 训练所添加的部分  
4) 解冻基网络的一些层  
5) 联合训练解冻的这些层和添加的部分  
特征提取的过程已经完成了前三个步骤，继续进行第四步：先解冻conv_base，然后冻结其中的部分层

In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

我们将微调最后三个卷积层，即直到block4_pool前的所有层都应该被冻结，而block5_conv1、block5_conv2、block5_conv3三层是可以被训练的。  
为什么不微调更多层？为什么不微调整个卷积基？可以这么做，但是需要考虑以下几点  
* 卷积基中更靠底部的层编码的是更加通用的可复用特征，而更靠顶部的层编码的是更专业化的特征。微调这些更专业化的特征更加游泳，因为它们需要在你的新问题上改变用途。微调更靠底部的层，得到的回报更少。
* 训练的参数越多，过拟合的风险越大。卷积基有1500万个参数，所以在小型数据集上训练这么多参数是有风险的。

#### 冻结直到block4_pool的所有层

In [13]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [14]:
# 检查下
for _layer in conv_base.layers:
    print(_layer.name, _layer.trainable)

input_2 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


#### 这里将使用学习率非常小的RMSProp优化器来实现。之所以让学习率很小，是因为对于微调的三层表示，我们希望其变化的范围不要太大。太大的权重更新可能会破坏这些表示

#### 执行“特征提取”中的代码，这里使用数据增强的代码

In [15]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)  # 将conv_base当作一层
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

W0821 10:53:54.646733 140736591090624 deprecation.py:506] From /Users/schp/miniconda2/envs/keras2.0.8/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1205: calling reduce_prod_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


如预训练中的文章中提到的一样，使用fit_generator进行训练并绘制结果